<a href="https://colab.research.google.com/github/Mohadesehzarei/SVM-RFECV/blob/main/SVM_RFECV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold,StratifiedKFold #for K-fold cross validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
import itertools


# Input data and class
X= pd.read_csv('input data')
y= pd.read_csv('input class')

# Split Dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65)
print(y_test)


#
model_Linear_SVM = svm.SVC(kernel='linear', probability=True)
rfecv = RFECV(estimator=model_Linear_SVM, step=1, cv=10,scoring='accuracy')   #10-fold cross-validation
rfecv = rfecv.fit(X_train, y_train)

#Number of classifiers
df=rfecv.n_features_
df1=X_train.columns[rfecv.support_]

#Accuracy of Model
scores = cross_val_score(rfecv, X_train, y_train, scoring='accuracy', cv=10, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


# Test set
y_test_scores_linear=rfecv.predict(X_test)

y_test_scores_linear1 = pd.DataFrame(y_test_scores_linear) 
df=classification_report(y_test,y_test_scores_linear)
print(df)



# plot_classification_report
def plot_classification_report(classificationReport,
                               title='Classification report',
                               cmap='RdBu'):

    classificationReport = classificationReport.replace('\n\n', '\n')
    classificationReport = classificationReport.replace(' / ', '/')
    lines = classificationReport.split('\n')

    classes, plotMat, support, class_names = [], [], [], []
    for line in lines[1:]:  # if you don't want avg/total result, then change [1:] into [1:-1]
        t = line.strip().split()
        if len(t) < 2:
            continue
        classes.append(t[0])
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        class_names.append(t[0])
        plotMat.append(v)

    plotMat = np.array(plotMat)
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup)
                   for idx, sup in enumerate(support)]

    plt.imshow(plotMat, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.title(title)
    plt.colorbar()
    plt.xticks(np.arange(3), xticklabels, rotation=45)
    plt.yticks(np.arange(len(classes)), yticklabels)

    upper_thresh = plotMat.min() + (plotMat.max() - plotMat.min()) / 10 * 8
    lower_thresh = plotMat.min() + (plotMat.max() - plotMat.min()) / 10 * 2
    for i, j in itertools.product(range(plotMat.shape[0]), range(plotMat.shape[1])):
        plt.text(j, i, format(plotMat[i, j], '.2f'),
                 horizontalalignment="center",
                 color="white" if (plotMat[i, j] > upper_thresh or plotMat[i, j] < lower_thresh) else "black")

    #plt.ylabel('Metrics')
    #plt.xlabel('Classes')
    plt.tight_layout()


sampleClassificationReport = """       precision    recall  f1-score   support
    """

plot_classification_report(sampleClassificationReport)
plt.savefig("CR_acute.jpg", dpi=300)

# plotting the confusion matrix
plot_confusion_matrix( rfecv, X_test, y_test)
plt.savefig("CM_acute.jpg", dpi=300)
plt.show()